In [66]:
# Librerías
import numpy as np
import math
import random
import sympy
import matplotlib.pyplot as plt
from numpy import pi,sin,cos,e,log,sqrt
from numpy import linalg as la
from matplotlib import animation
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits import mplot3d
from matplotlib import cm
from matplotlib.ticker import LinearLocator
from PIL import Image, ImageOps
import os
import cv2
import random

#1. Generar una malla
h = 0.025
Nx = int(10/h)
delta = np.linspace(0, 10, Nx)#Nx=400

#2.Discretizar en cada punto de la malla los siguientes parámetros:
#desidad
Rho = 1.25

#velocidad de onda
Beta = 1.4 

#constante elástica
Mu = Rho*(Beta)**2

#3. discretización temporal para 5 segundos
dt = 0.01
N1 = int(5/dt)
T = np.linspace(0, 5, N1) #N1=500

#4.Sobre la dizcretizacion temporal generar el impulso de Ricker 
Ricker = [0]*N1
def dis_Temp(t):

    tp = 1/5
    a = ( ((pi/3) * (t - 2*tp))/tp )**2  
    return (a - 0.5 ) * np.exp(-a) 

for i in range(Nx):  
    Ricker[i] = dis_Temp(T[i])
    
#5 Generar una función para las fronteras
def frontera(): 
    r = 1
    K = 0.15
    d =  np.ones((Nx, Nx))
 
    for i in range(Nx):
        for j in range(Nx):           
            if(i < 19):
                r = (19 - i)  
            elif(19 <= i <= Nx and j<19):
                r = (19 - j)
            elif(19 <= i <= Nx and (Nx + 1 - 19) <= j <= Nx):
                r =(19 + j - (Nx + 1))
            elif((Nx + 1 - 19) < i <= Nx and 19 <= j <= Nx-19):
                r = (19 + i - (Nx + 1))      
            if (r != 1):
                d[i][j] = np.exp( -1 * (K * r)**2 )
                r = 1          
    return d
   
v_D = frontera()

#Derivadas

#derivada centrada con respecto a x 
def dercent(M, N, pp): 
    Dx = np.zeros((N,N))   
    if pp==0:
      for x in range(N):
          for z in range(N):
                if z==0:
                    dx =(M[1,x] - M[0,x] )/(h)  
                    
                elif z>=1:
                    dx =(M[z,x] - M[z-1,x] )/(h)
                Dx[z][x] = dx
                
    else:
      for x in range(N):
          for z in range(N):
                if z==0:
                    dz =(M[x,1] - M[x,0] )/(h)   
                    
                elif z>=1:
                    dz =(M[x,z] - M[x,z-1] )/(h)
                Dx[x,z] = dz         
    return Dx

# Derivada progresiva
def derpro(M, N, pp):  
    Dx = np.zeros((N,N))   
    if pp==0:
      for x in range(N):              
          for z in range(N):
                if z<=N-2:
                    dx = (M[z+1,x] - M[z,x])/(h)            
                    
                else:
                    dx = (M[z,x] - M[z-1,x])/(h)
                Dx[z][x] = dx
                    
    else: 
      for x in range(N):   
          for z in range(N):
                if z<=N-2:
                    dz = (M[x, z+1] - M[x, z])/(h)   
                    
                else:
                    dz = (M[x, z] - M[x, z-1])/(h)                  
                Dx[x,z] = dz
    return Dx

#Paso 6. inicializar en cero para t = 0 las siguintes matrices en 0
    
Vxt = np.zeros((N1,Nx, Nx))
m_Oxy = np.zeros((Nx, Nx))
m_Oyz = np.zeros((Nx, Nx))

dxVx = np.zeros((Nx, Nx))
dzVx = np.zeros((Nx, Nx))

sigxy = np.zeros((Nx, Nx))
sigyz = np.zeros((Nx, Nx))

#Algunas matrices
final =  np.zeros((N1, Nx, Nx))

v_xy =  np.zeros((N1, Nx, Nx))
v_xz =  np.zeros((N1, Nx, Nx))

v_xy[0, :, :] = m_Oxy
v_xz[0, :, :] = m_Oyz

temp_Vx = np.zeros((2, Nx, Nx))

#Paso 7
#Inicio del tiempo
for t in range(1,len(T)):
#Se agrega el pulso de riker
    for i in range (3):
        for j in range (3):
            s_i = int((Nx/2)-(2-i))
            s_j = int((Nx/2)-(2-j))     
            Vxt[t-1][s_i][s_j] = Vxt[t-1][s_i][s_j] + Ricker[t-1]
#Se calculan las diferencias finitas centradas
    dxVx = dercent(Vxt[t-1,:,:],Nx, 0)
    dzVx = dercent(Vxt[t-1,:,:],Nx, 1)
#Se calculan los esfuerzos       
    for i in range (Nx):
        for j in range (Nx):     
            v_xy[t, i, j] = v_xy[t-1, i, j] + dt * Mu * dxVx[i][j]
            v_xy[t, i, j] = v_xy[t, i, j]*v_D[i][j]
                
            v_xz[t, i, j] = v_xz[t-1, i, j] + dt * Mu * dzVx[i][j]
            v_xz[t, i, j] = v_xz[t, i, j] *v_D[i][j]
        
#Se calculan las derivadas progresivas               
    sigxy = derpro(v_xy[t, :, :],Nx, 0)
    sigyz = derpro(v_xz[t, :, :],Nx, 1)

#Se actualizan las velocidades y se multiplican por la frontera absorvente 
    for i in range (Nx):
        for j in range (Nx):
            Vxt[t][i][j] = Vxt[t-1][i][j] + (dt/Rho) *( (sigxy[i][j]) + (sigyz[i][j]))
            Vxt[t][i][j] = Vxt[t][i][j] * v_D[i][j]
            final[t, i, j] = Vxt[t][i][j]

#Paso 8. Se crean las imagenes y se guardan en la pc
fig = plt.figure()
ax = plt.axes(projection='3d')
xv, yv = np.meshgrid(delta,delta)
for t in range(0,len(T),5):
    ax.plot_surface(xv, yv, final[t,:,:],cmap=cm.coolwarm,linewidth=0, antialiased=False)
    ax.set_xlabel("Eje x")
    ax.set_ylabel("Eje y")
    plt.savefig("C:/Users/braya/Desktop/Matematicas/Segundo Semestre 2021/Computación y Métodos Númericos/Proyectos/Ultimo proyecto/Codigo/" + str(t))
    
#Paso 8. Se monta la animacion usando las imagenes recien calculadas
#Leer imagenes
img_array = []
for k in range(0,len(T),5):
    img=cv2.imread(str(k)+ ".png")
    img_array.append(img)
#Tamaño de la última imagen alto y ancho
height, width  = img.shape[:2]

#Caracteristicas del video
video = cv2.VideoWriter('Video.mp4',cv2.VideoWriter_fourcc(*'mp4v'),7,(width,height))

# Poner cada frame en video
for k in range(0,len(img_array)):
    video.write(img_array[k])

#liberar recursos
video.release()

#Paso 9. Se generan los sismogramas
# prints a random value from the list
opgg=list()
for t in range(0,Nx):
    opgg.append(t)
for t in range(0,10):
    fig = plt.figure()
    rr=random.choice(opgg)
    rrr=random.choice(opgg)
    plt.plot(T,final[:,rr,rrr])
    plt.title("Sismograma para i=" + str(rr) +" j=" + str(rrr))
    plt.xlabel("Tiempo ")
    plt.show()

#Sismograma
opgg=list()
for t in range(0,Nx):
    opgg.append(t)
fig = plt.figure()
for t in range(0,10):
    rr=random.choice(opgg)
    rrr=random.choice(opgg)
    plt.plot(T,final[:,rr,rrr],"b")
    plt.title("Sismograma")
    plt.xlabel("Tiempo ")
plt.show()

#Impluso de Ricker
fig = plt.figure()
plt.plot(T,Ricker)
plt.title("Impulso de Ricker")
plt.xlabel("Tiempo")
plt.show()